# Part 2 Inference

## 1. Import data

In [6]:
import pandas as pd
import torch
import torch.nn.functional as F
import glob
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from pathlib import Path
from tqdm import tqdm # 進度條套件


# For colab
# from google.colab import drive
# drive.mount('/content/drive')
# news_file_path = '/content/drive/MyDrive/fiisual/news_data2/*.csv'
# model_name = "/content/drive/MyDrive/fiisual/my_finbert_model_backup"

# 1. 準備工作
# 假設你的資料在 df_news，欄位有 ['Date', 'Title']

current_dir = Path.cwd()
project_root = current_dir.parent
news_dir = project_root / 'data' / 'news_data'
news_file_path = str(news_dir / '*.csv')

# 2. 抓取所有 CSV 檔案的清單
all_files = glob.glob(news_file_path)
print(f"📂 找到 {len(all_files)} 個 CSV 檔案")

# 3. 讀取並合併
df_list = []

for filename in all_files:
    try:
        # 讀取單個 CSV
        df = pd.read_csv(filename, index_col=None, header=0)
        df_list.append(df)
    except Exception as e:
        print(f"⚠️ 無法讀取 {filename}: {e}")

# 使用 concat 一次性合併
if df_list:
    df_news = pd.concat(df_list, axis=0, ignore_index=True)
    print(f"✅ 合併完成！總共有 {len(df_news)} 筆資料")

    # 4. 基本檢查
    # 去除重複 (以防爬蟲有重疊抓取)
    if 'title' in df_news.columns:
        initial_count = len(df_news)
        df_news = df_news.drop_duplicates(subset=['title'])
        print(f"🧹 去除重複後剩餘: {len(df_news)} 筆 (刪除了 {initial_count - len(df_news)} 筆)")

    # 檢查欄位
    print(df_news.head())
else:
    print("❌ 沒有讀取到任何資料，請檢查路徑是否正確。")
    


📂 找到 32 個 CSV 檔案
✅ 合併完成！總共有 3196 筆資料
🧹 去除重複後剩餘: 2128 筆 (刪除了 1068 筆)
         date                                              title  \
0  2025-11-13  Repsol explores reverse merger for upstream un...   
1  2025-11-13  Google to invest $40 billion in Texas data cen...   
2  2025-11-13  Stocks Settle Mixed as Fed Comments Suggest a ...   
3  2025-11-13  Samsung raised memory chip prices by up to 60%...   
4  2025-11-13  Trump ready to exempt some food products from ...   

           media                                               link  
0  Yahoo Finance  https://finance.yahoo.com/news/repsol-explores...  
1  Yahoo Finance  https://finance.yahoo.com/news/google-invest-4...  
2  Yahoo Finance  https://finance.yahoo.com/news/stocks-settle-m...  
3  Yahoo Finance  https://finance.yahoo.com/news/samsung-raised-...  
4  Yahoo Finance  https://finance.yahoo.com/news/trump-ready-exe...  


## 2. Inference

In [27]:
model_dir = project_root / 'models' / 'finbert_sentiment'
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSequenceClassification.from_pretrained(model_dir)

# 如果有 GPU 就用 GPU，不然用 CPU
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"🚀 使用 NVIDIA GPU 加速: {torch.cuda.get_device_name(0)}")

# 其次檢查 Mac Apple Silicon (MPS)
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print("🍎 使用 Apple Silicon (Metal) 加速")

# 最後不得已才用 CPU
else:
    device = torch.device("cpu")
    print("⚠️ 未偵測到 GPU，將使用 CPU 執行 (速度較慢)")

model.to(device)
print(f"🚀 使用裝置: {device}")

# 2. 定義推論函數 (支援 Batch)
def predict_sentiment_label_batch(titles, batch_size=32):
    model.eval()
    all_labels = []

    # 使用 tqdm 顯示進度條 (如果沒裝 tqdm 可以拿掉這行，改用一般 range)
    for i in range(0, len(titles), batch_size):
        batch = titles[i : i+batch_size]

        # 1. Tokenize
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=64)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # 2. Inference
        with torch.no_grad():
            outputs = model(**inputs)

        # 3. 計算機率 (Softmax)
        # outputs.logits 形狀是 [batch_size, 3] -> 分別對應 [Negative, Neutral, Positive]
        probs = F.softmax(outputs.logits, dim=-1)

        # 4. 取最大值的索引 (Argmax) -- 這就是關鍵修改！
        # 找出每一列中機率最大的那個 index (0, 1, 或 2)
        batch_labels = torch.argmax(probs, dim=-1).cpu().numpy()

        all_labels.extend(batch_labels)

    return all_labels

def predict_sentiment_batch(titles, batch_size=32):
    model.eval()
    scores = []

    # 使用 tqdm 顯示進度條
    for i in range(0, len(titles), batch_size):
        batch = titles[i : i+batch_size]

        # Tokenize
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=64)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Inference
        with torch.no_grad():
            outputs = model(**inputs)

        # 計算機率 (Softmax)
        probs = F.softmax(outputs.logits, dim=-1)

        # 計算分數: Positive(2) - Negative(0)
        # FinBERT label order: [0: Negative, 1: Neutral, 2: Positive]
        batch_scores = (probs[:, 2] - probs[:, 0]).cpu().numpy()
        scores.extend(batch_scores)

    return scores

# 3. 執行推論
print("🧠 開始進行情緒推論...")
# 為了避免標題不是字串格式導致報錯，先轉型
titles_list = df_news['title'].astype(str).tolist()
df_news['Sentiment_Score'] = predict_sentiment_batch(titles_list)
df_news['Sentiment_Label'] = predict_sentiment_label_batch(titles_list)

print("✅ 推論完成！前 5 筆預覽：")
print(df_news[['date', 'title', 'Sentiment_Label', 'Sentiment_Score']].head())

⚠️ 未偵測到 GPU，將使用 CPU 執行 (速度較慢)
🚀 使用裝置: cpu
🧠 開始進行情緒推論...
✅ 推論完成！前 5 筆預覽：
         date                                              title  \
0  2025-11-13  Repsol explores reverse merger for upstream un...   
1  2025-11-13  Google to invest $40 billion in Texas data cen...   
2  2025-11-13  Stocks Settle Mixed as Fed Comments Suggest a ...   
3  2025-11-13  Samsung raised memory chip prices by up to 60%...   
4  2025-11-13  Trump ready to exempt some food products from ...   

   Sentiment_Label  Sentiment_Score  
0                1         0.230534  
1                1         0.219046  
2                0        -0.862863  
3                2         0.976522  
4                1         0.256578  


None
